In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [3]:
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your openai API key: ")

In [20]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [11]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")



In [21]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)

In [22]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [32]:
schema = {
  "type": "object",
  "properties": {
    "srcdb": { "type": "string" },
    "count": { "type": "integer" },
    "results": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "key": { "type": "string" },
          "code": { "type": "string" },
          "title": { "type": "string" },
          "crn": { "type": "string" },
          "no": { "type": "string" },
          "total": { "type": "string" },
          "schd": { "type": "string" },
          "stat": { "type": "string" },
          "hide": { "type": "string" },
          "isCancelled": { "type": "string" },
          "meets": { "type": "string" },
          "mpkey": { "type": "string" },
          "meetingTimes": { "type": "string" }, 
          "instr": { "type": "string" },
          "start_date": { "type": "string", "format": "date" },
          "end_date": { "type": "string", "format": "date" },
          "permreq": { "type": "string" },
          "rpt": { "type": "string" },
          "cart_opts": { "type": "string" }, 
          "linked_crns": { "type": "string" },
          "rfam": { "type": "string" },
          "lrfam": { "type": "string" },
          "srcdb": { "type": "string" },
          "changehash": { "type": "string" },
          "offsets": { "type": "string" },
          "rank": { "type": "integer" }
        },
        "required": [
          "key", "code", "title", "crn", "no", "total", "schd", "stat", 
          "instr", "start_date", "end_date", "srcdb", "rank"
        ]
      }
    }
  },
  "required": ["srcdb", "count", "results"]
}
import json
encoded_data = json.dumps(schema).encode("utf-8") 

In [ ]:
# Load and chunk contents of the blog
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://cs2240.graphics")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [35]:
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [36]:
response = graph.invoke({"question": "What is the course about?"})
print(response["answer"])

The course is focused on computer graphics, guiding students from basic knowledge to the ability to read and implement graphics research papers. Key topics include light transport physics, geometry representation, quadratic optimization, and continuum mechanics for simulating solid objects. By the end, students will be prepared for research careers or roles in industrial labs within the field of computer graphics.


In [37]:
response = graph.invoke({"question": "Who is the instructor?"})
print(response["answer"])

The instructor is Daniel Ritchie.


In [38]:
response = graph.invoke({"question": "If i want to pursue a career as a AI Enginner in graphics, is this course good?"})
print(response["answer"])

Yes, this course is a solid choice for pursuing a career as an AI Engineer in graphics. It offers advanced knowledge in key areas like rendering, geometry processing, and optimization, preparing you for both academic and industrial research roles. Additionally, the course emphasizes practical project work, which is essential for a successful career in this field.
